In [ ]:
# !pip install osmnx geopandas shapely rtree pyproj pandas tqdm

In [39]:
import os
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from tqdm import tqdm

import osmnx as ox
# configure osmnx
# Speed & caching settings
ox.settings.use_cache = True
ox.settings.cache_folder = "./osm_cache"
ox.settings.log_console = False
ox.settings.timeout = 180
# ox.settings.overpass_endpoint = "https://overpass.kumi.systems/api/interpreter"
ox.settings.overpass_endpoint = "https://overpass.private.coffee/api/interpreter"

In [27]:
stations = [
    ("Anand Vihar, Delhi (DPCC / CPCB)", 28.6559, 77.2949, "CPCB list"),
    ("Punjabi Bagh, Delhi (DPCC)", 28.674, 77.131, "CPCB list"),
    ("Mandir Marg, Delhi (DPCC)", 28.636429, 77.201067, "GIS study"),
    ("R K Puram, Delhi (DPCC)", 28.563262, 77.186937, "paper"),
    ("Sector-51 Gurugram - HSPCB", 28.43518, 77.072, "env clearance doc"),
    ("Vikas Sadan Gurugram - HSPCB", 28.450129, 77.026306, "amazon link?"),
    ("Sector - 125 Noida - UPPCB", 28.5897, 77.31, "UPPCB"),
]

df_stations = pd.DataFrame(stations, columns=["station", "lat", "lon", "notes"])
gdf_stations = gpd.GeoDataFrame(df_stations,
                                geometry=gpd.points_from_xy(df_stations.lon, df_stations.lat),
                                crs="EPSG:4326")


In [28]:
df_stations

,station,lat,lon,notes
0,"Anand Vihar, Delhi (DPCC / CPCB)",28.655900,77.294900,CPCB list
1,"Punjabi Bagh, Delhi (DPCC)",28.674000,77.131000,CPCB list
2,"Mandir Marg, Delhi (DPCC)",28.636429,77.201067,GIS study
3,"R K Puram, Delhi (DPCC)",28.563262,77.186937,paper
4,Sector-51 Gurugram - HSPCB,28.435180,77.072000,env clearance doc
5,Vikas Sadan Gurugram - HSPCB,28.450129,77.026306,amazon link?
6,Sector - 125 Noida - UPPCB,28.589700,77.310000,UPPCB


In [29]:
gdf_stations

,station,lat,lon,notes,geometry
0,"Anand Vihar, Delhi (DPCC / CPCB)",28.655900,77.294900,CPCB list,POINT (77.2949 28.6559)
1,"Punjabi Bagh, Delhi (DPCC)",28.674000,77.131000,CPCB list,POINT (77.131 28.674)
2,"Mandir Marg, Delhi (DPCC)",28.636429,77.201067,GIS study,POINT (77.20107 28.63643)
3,"R K Puram, Delhi (DPCC)",28.563262,77.186937,paper,POINT (77.18694 28.56326)
4,Sector-51 Gurugram - HSPCB,28.435180,77.072000,env clearance doc,POINT (77.072 28.43518)
5,Vikas Sadan Gurugram - HSPCB,28.450129,77.026306,amazon link?,POINT (77.02631 28.45013)
6,Sector - 125 Noida - UPPCB,28.589700,77.310000,UPPCB,POINT (77.31 28.5897)


In [ ]:
# !pip install pyproj

In [47]:
# import shapely
# p = shapely.Point(77.2949, 28.6559)  # Anand Vihar (lon, lat)
# buf = make_buffer(p, meters=10)
# print(buf.bounds)

In [48]:
def get_utm_crs_for_point(lat, lon):
    """Return an appropriate UTM CRS for given lat/lon using osmnx utility."""
    try:
        # osmnx returns pyproj CRS
        return ox.projection.get_utm_crs(lat, lon)
    except Exception:
        # fallback: approximate by EPSG:3857 for metric operations (less accurate near poles, ok here)
        import pyproj
        return pyproj.CRS.from_epsg(3857)

# def make_buffer(point_geom, meters=500):
#     """
#     Create a buffer polygon (in WGS84 returned) around a point geometry with radius in meters.
#     Steps: project to appropriate UTM -> buffer -> reproject to WGS84.
#     """
#     lat, lon = point_geom.y, point_geom.x
#     utm_crs = get_utm_crs_for_point(lat, lon)
#     # project point to utm
#     g = gpd.GeoSeries([point_geom], crs="EPSG:4326").to_crs(utm_crs)
#     buf = g.buffer(meters).iloc[0]
#     buf_wgs84 = gpd.GeoSeries([buf], crs=utm_crs).to_crs("EPSG:4326").iloc[0]
#     return buf_wgs84

def make_buffer(point_geom, meters=500):
    """
    Create a small, valid buffer polygon (WGS84) around a point.
    Works robustly for OSMnx / Overpass.
    """
    lat, lon = point_geom.y, point_geom.x
    utm_crs = get_utm_crs_for_point(lat, lon)
    
    # project to local UTM, buffer, and reproject back
    point_gdf = gpd.GeoSeries([point_geom], crs="EPSG:4326").to_crs(utm_crs)
    buffer_geom = point_gdf.buffer(meters).to_crs("EPSG:4326").iloc[0]

    # Clean geometry and force validity
    buffer_geom = buffer_geom.buffer(0)
    if not buffer_geom.is_valid:
        buffer_geom = buffer_geom.buffer(0)
    
    # Clip insane coordinates just in case (fixes overpass max area issue)
    minx, miny, maxx, maxy = buffer_geom.bounds
    if (maxx - minx) > 0.05 or (maxy - miny) > 0.05:  # ~5 km threshold
        raise ValueError(f"Buffer for {point_geom} too large: {maxx-minx:.4f}, {maxy-miny:.4f}")

    return buffer_geom


def area_m2(geom):
    """Return approximate area in m^2 by projecting geometry to UTM chosen for centroid."""
    lat, lon = geom.centroid.y, geom.centroid.x
    utm_crs = get_utm_crs_for_point(lat, lon)
    g = gpd.GeoSeries([geom], crs="EPSG:4326").to_crs(utm_crs)
    return g.iloc[0].area

In [62]:
# ---------------------------
# 3) Road / traffic features via OSM
# ---------------------------
major_highways = ['motorway','trunk','primary','secondary','tertiary']

def compute_road_features(point_row, buffer_poly):
    """
    point_row: a Series with columns ['station','lat','lon','geometry']
    buffer_poly: shapely polygon in WGS84
    Returns:
      dict with distance_to_major_road (m), total_road_length_m, major_road_length_m
    """
    # bbox for Overpass query (use a small bbox around the buffer to limit query)
    minx, miny, maxx, maxy = buffer_poly.bounds
    bbox = (minx, miny, maxx, maxy)  # west, south, east, north
    tags = {'highway': True}
    print(f"Trying with BBOX: {bbox} and tags: {tags}")

    # Query OSM for highway features within bbox
    try:
        ways = ox.features_from_bbox(bbox, tags)
    except Exception as e:
        # print error and return NaNs / zeros
        print(f"OSM roads fetch failed for {point_row.station}: {e}")
        return {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    # Keep only geometries that are linestring-like
    if ways.empty or 'geometry' not in ways.columns:
        return {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    ways = ways[ways.geometry.notnull()].copy()
    # if 'highway' column missing, try other representations
    if 'highway' not in ways.columns:
        # sometimes the highway tag is in 'tags' or dtype object - attempt to extract
        ways['highway'] = ways.apply(lambda r: r.get('highway') if isinstance(r, dict) else None, axis=1)

    # explode multi-geometries
    ways = ways.explode(ignore_index=True)
    ways = ways[ways.geometry.type.isin(['LineString','MultiLineString'])]
    if ways.empty:
        return {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    # Intersect each geometry with buffer polygon
    ways['segment'] = ways.geometry.intersection(buffer_poly)
    ways = ways[~ways['segment'].is_empty].copy()
    if ways.empty:
        return {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    # Project to UTM to measure length
    utm = get_utm_crs_for_point(point_row.lat, point_row.lon)
    segs = gpd.GeoSeries(ways['segment'].values, crs="EPSG:4326").set_crs("EPSG:4326").to_crs(utm)
    segs = segs[segs.notnull()]
    lengths = segs.length
    total_len = lengths.sum()

    # identify major segments by highway tag
    # highway tags can be strings or lists; convert and match
    def is_major_tag(val):
        if val is None:
            return False
        s = str(val).lower()
        return any(h in s for h in major_highways)

    # align list lengths
    # note: ways['highway'] can be scalar per feature, repeating to same index as segs
    ways = ways.reset_index(drop=True).loc[segs.index if hasattr(segs, "index") else ways.index]
    ways = ways.assign(length_m=lengths.values)
    ways['is_major'] = ways['highway'].apply(is_major_tag)
    major_len = ways.loc[ways['is_major'], 'length_m'].sum() if 'length_m' in ways.columns else 0.0

    # compute distance from station to nearest major road (projected)
    major_segs = gpd.GeoSeries(ways.loc[ways['is_major'], 'segment'].values,
                               crs="EPSG:4326").to_crs(utm)
    if not major_segs.empty:
        station_point_proj = gpd.GeoSeries([point_row.geometry], crs="EPSG:4326").to_crs(utm).iloc[0]
        dists = major_segs.distance(station_point_proj)
        min_dist = float(dists.min())
    else:
        min_dist = np.nan

    return {'distance_to_major_road': float(min_dist) if np.isfinite(min_dist) else np.nan,
            'total_road_length_m': float(total_len),
            'major_road_length_m': float(major_len)}


In [63]:

# ---------------------------
# 4) Land use features via OSM (proxy for LULC)
# ---------------------------
def compute_landuse_features(buffer_poly):
    """
    buffer_poly: shapely polygon in WGS84
    Returns: pct_green, pct_industrial, pct_residential (fractions 0..1)
    NOTE: OSM landuse is a proxy; for authoritative LULC, replace with raster zonal stats.
    """
    minx, miny, maxx, maxy = buffer_poly.bounds
    bbox = (minx, miny, maxx, maxy)  # west, south, east, north
    tags = {'landuse': True, 'leisure': True, 'natural': True}
    print(f"Trying with BBOX: {bbox} and tags: {tags}")
    
    try:
        polys = ox.features_from_bbox(bbox, tags)
    except Exception as e:
        print("OSM landuse fetch failed:", e)
        return {'pct_green': np.nan, 'pct_industrial': np.nan, 'pct_residential': np.nan}

    if polys.empty or 'geometry' not in polys.columns:
        return {'pct_green': 0.0, 'pct_industrial': 0.0, 'pct_residential': 0.0}

    polys = polys[polys.geometry.notnull()].explode(ignore_index=True)
    polys = polys[polys.geometry.type.isin(['Polygon','MultiPolygon'])].copy()
    if polys.empty:
        return {'pct_green': 0.0, 'pct_industrial': 0.0, 'pct_residential': 0.0}

    # compute intersection with buffer
    polys['intersect'] = polys.geometry.intersection(buffer_poly)
    polys = polys[~polys['intersect'].is_empty].copy()
    if polys.empty:
        return {'pct_green': 0.0, 'pct_industrial': 0.0, 'pct_residential': 0.0}

    # project to UTM for area calc
    centroid = buffer_poly.centroid
    utm = get_utm_crs_for_point(centroid.y, centroid.x)
    intersect_gs = gpd.GeoSeries(polys['intersect'].values, crs="EPSG:4326").to_crs(utm)
    areas = intersect_gs.area
    polys = polys.reset_index(drop=True).assign(area_m2=areas.values)
    total_area = polys['area_m2'].sum() if len(polys)>0 else 0.0
    if total_area <= 0:
        return {'pct_green': 0.0, 'pct_industrial': 0.0, 'pct_residential': 0.0}

    def area_by_keywords(keywords):
        mask = polys.apply(lambda r: any((kw in str(r.get('landuse','')).lower()) for kw in keywords) or
                                       any((kw in str(r.get('leisure','')).lower()) for kw in keywords) or
                                       any((kw in str(r.get('natural','')).lower()) for kw in keywords),
                           axis=1)
        return polys.loc[mask, 'area_m2'].sum()

    green_area = area_by_keywords(['forest','grass','park','recreation','garden','meadow','wood','green'])
    industrial_area = area_by_keywords(['industrial','quarry','landfill'])
    residential_area = area_by_keywords(['residential','housing','residential;apartments'])  # some tag variants

    return {'pct_green': float(green_area/total_area),
            'pct_industrial': float(industrial_area/total_area),
            'pct_residential': float(residential_area/total_area)}


In [64]:
# ---------------------------
# 5) Building morphology via OSM footprints
# ---------------------------
def compute_building_features(buffer_poly):
    """
    Returns building_density (fraction of buffer area that is building footprint),
    avg_building_area_m2, median_building_area_m2, building_count
    """
    minx, miny, maxx, maxy = buffer_poly.bounds
    bbox = (minx, miny, maxx, maxy)  # west, south, east, north
    tags = {'building': True}
    print(f"Trying with BBOX: {bbox} and tags: {tags}")

    try:
        blds = ox.features_from_bbox(bbox, tags)
    except Exception as e:
        print("OSM buildings fetch failed:", e)
        return {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    if blds.empty or 'geometry' not in blds.columns:
        return {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    blds = blds[blds.geometry.notnull()].explode(ignore_index=True)
    blds = blds[blds.geometry.type.isin(['Polygon','MultiPolygon'])].copy()
    if blds.empty:
        return {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    blds['intersect'] = blds.geometry.intersection(buffer_poly)
    blds = blds[~blds['intersect'].is_empty].copy()
    if blds.empty:
        return {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    centroid = buffer_poly.centroid
    utm = get_utm_crs_for_point(centroid.y, centroid.x)
    areas = gpd.GeoSeries(blds['intersect'].values, crs="EPSG:4326").to_crs(utm).area
    blds = blds.reset_index(drop=True).assign(area_m2=areas.values)
    total_bld_area = blds['area_m2'].sum()
    buf_area = area_m2(buffer_poly)
    building_density = float(total_bld_area / buf_area) if buf_area>0 else 0.0

    return {'building_density': building_density,
            'avg_building_area_m2': float(blds['area_m2'].mean()),
            'median_building_area_m2': float(blds['area_m2'].median()),
            'building_count': int(len(blds))}

In [65]:

# ---------------------------
# 6) Pipeline: run for each station
# ---------------------------
results = []
buffer_radius_m = 100  # change to desired radius (meters)

print("Starting hyperlocal extraction for", len(gdf_stations), "stations. Buffer radius:", buffer_radius_m, "m")
for idx, row in tqdm(gdf_stations.reset_index(drop=True).iterrows(), total=len(gdf_stations)):
    station_name = row['station']
    pt = row.geometry
    try:
        buffer_poly = make_buffer(pt, meters=buffer_radius_m)
    except Exception as e:
        print(f"Failed to build buffer for {station_name}: {e}")
        buffer_poly = None

    if buffer_poly is None or buffer_poly.is_empty:
        # append empty features
        out = dict(row.drop(labels=['geometry']).to_dict())
        out.update({
            'distance_to_major_road': np.nan,
            'total_road_length_m': 0.0,
            'major_road_length_m': 0.0,
            'pct_green': np.nan,
            'pct_industrial': np.nan,
            'pct_residential': np.nan,
            'building_density': 0.0,
            'avg_building_area_m2': np.nan,
            'median_building_area_m2': np.nan,
            'building_count': 0
        })
        results.append(out)
        continue

    # Roads
    try:
        road_feats = compute_road_features(row, buffer_poly)
    except Exception as e:
        print(f"Error computing road features for {station_name}: {e}")
        road_feats = {'distance_to_major_road': np.nan, 'total_road_length_m': 0.0, 'major_road_length_m': 0.0}

    # Landuse
    try:
        land_feats = compute_landuse_features(buffer_poly)
    except Exception as e:
        print(f"Error computing landuse for {station_name}: {e}")
        land_feats = {'pct_green': np.nan, 'pct_industrial': np.nan, 'pct_residential': np.nan}

    # Buildings
    try:
        bld_feats = compute_building_features(buffer_poly)
    except Exception as e:
        print(f"Error computing buildings for {station_name}: {e}")
        bld_feats = {'building_density': 0.0, 'avg_building_area_m2': np.nan, 'median_building_area_m2': np.nan, 'building_count': 0}

    out = dict(row.drop(labels=['geometry']).to_dict())
    out.update(road_feats)
    out.update(land_feats)
    out.update(bld_feats)
    results.append(out)

    # polite pause to avoid hammering Overpass (optional)
    time.sleep(1.0)

Starting hyperlocal extraction for 7 stations. Buffer radius: 100 m


  0%|          | 0/7 [00:00<?, ?it/s]

Trying with BBOX: (77.29400168471588, 28.655111711426496, 77.29579831528412, 28.656688282646677) and tags: {'highway': True}
Trying with BBOX: (77.29400168471588, 28.655111711426496, 77.29579831528412, 28.656688282646677) and tags: {'landuse': True, 'leisure': True, 'natural': True}
Trying with BBOX: (77.29400168471588, 28.655111711426496, 77.29579831528412, 28.656688282646677) and tags: {'building': True}


 14%|█▍        | 1/7 [00:18<01:51, 18.59s/it]

Trying with BBOX: (77.13010168471587, 28.673211847551748, 77.1318983152841, 28.67478814651903) and tags: {'highway': True}
Trying with BBOX: (77.13010168471587, 28.673211847551748, 77.1318983152841, 28.67478814651903) and tags: {'landuse': True, 'leisure': True, 'natural': True}
Trying with BBOX: (77.13010168471587, 28.673211847551748, 77.1318983152841, 28.67478814651903) and tags: {'building': True}


...

 71%|███████▏  | 5/7 [17:06<06:04, 182.30s/it]

Trying with BBOX: (77.02540768471587, 28.449339169412152, 77.02720431528411, 28.450918824688483) and tags: {'highway': True}
Trying with BBOX: (77.02540768471587, 28.449339169412152, 77.02720431528411, 28.450918824688483) and tags: {'landuse': True, 'leisure': True, 'natural': True}
OSM landuse fetch failed: No matching features. Check query location, tags, and log.
Trying with BBOX: (77.02540768471587, 28.449339169412152, 77.02720431528411, 28.450918824688483) and tags: {'building': True}


 86%|████████▌ | 6/7 [19:30<02:49, 169.13s/it]

Trying with BBOX: (77.30910168471587, 28.588911214224186, 77.3108983152841, 28.5904887798578) and tags: {'highway': True}
Trying with BBOX: (77.30910168471587, 28.588911214224186, 77.3108983152841, 28.5904887798578) and tags: {'landuse': True, 'leisure': True, 'natural': True}
Trying with BBOX: (77.30910168471587, 28.588911214224186, 77.3108983152841, 28.5904887798578) and tags: {'building': True}


100%|██████████| 7/7 [21:46<00:00, 186.64s/it]


In [70]:

# Build results DataFrame & some derived features
df_results = pd.DataFrame(results)
# derive fraction of major roads
df_results['major_road_fraction'] = df_results.apply(
    lambda r: float(r.major_road_length_m / r.total_road_length_m) if (r.total_road_length_m and not np.isnan(r.total_road_length_m)) else np.nan,
    axis=1
)

# Save CSV
# out_dir = '/mnt/data'
# os.makedirs(out_dir, exist_ok=True)
# out_csv = os.path.join(out_dir, 'delhi_hyperlocal_context_features.csv')
df_results.to_csv('delhi_hyperlocal_context_features.csv', index=False)
print("Saved results to: delhi_hyperlocal_context_features.csv")

Saved results to: delhi_hyperlocal_context_features.csv


In [66]:
results

[{'station': 'Anand Vihar, Delhi (DPCC / CPCB)',
  'lat': 28.6559,
  'lon': 77.2949,
  'notes': 'CPCB list',
  'distance_to_major_road': nan,
  'total_road_length_m': 407.38167952783743,
  'major_road_length_m': 0.0,
  'pct_green': 0.0038466799783292696,
  'pct_industrial': 0.0,
  'pct_residential': 0.21924256434370418,
  'building_density': 0.6140248660724802,
  'avg_building_area_m2': 6419.72922278817,
  'median_building_area_m2': 108.41661903221998,
  'building_count': 3},
 {'station': 'Punjabi Bagh, Delhi (DPCC)',
  'lat': 28.674,
  'lon': 77.131,
  'notes': 'CPCB list',
  'distance_to_major_road': nan,
  'total_road_length_m': 774.7269562412803,
  'major_road_length_m': 0.0,
  'pct_green': 0.0,
  'pct_industrial': 0.0,
  'pct_residential': 1.0,
  'building_density': 0.4170334408403472,
  'avg_building_area_m2': 272.5095019543239,
  'median_building_area_m2': 200.1124696087577,
  'building_count': 48},
 {'station': 'Mandir Marg, Delhi (DPCC)',
  'lat': 28.636429,
  'lon': 77.201067